In [1]:
import socket
import pandas as pd
import subprocess
from cassandra.cluster import Cluster
from time import sleep
from sqlalchemy           import create_engine, text
from sqlalchemy.orm       import sessionmaker
from sqlalchemy           import URL
from pymongo.mongo_client import MongoClient
from pymongo.server_api   import ServerApi

### Conectando ao Cassandra

In [2]:
hostname = socket.gethostname()
ip_address = socket.gethostbyname(hostname)
port = 9042

cluster = Cluster([ip_address], port=port)
session = cluster.connect('cc6240')

### Funções

#### Create

In [19]:
def criar_tabelas():
    # [Exemplo] student e instructor
    session.execute("CREATE TABLE instructor (name text, dept_name text, salary double, primary key((name), dept_name));")
    session.execute("CREATE TABLE student (name text, dept_name text, tot_cred tinyint, advisor text, primary key((name), dept_name));")
    
    # course_department
    session.execute("CREATE TABLE course_department (dept_name text, building text, budget text, course_id text, title text, credits text, PRIMARY KEY (dept_name, course_id, title));")

    # course
    session.execute("CREATE TABLE course (course_id text, title text, dept_name text, credits text, PRIMARY KEY((course_id), dept_name));")
    
    # section
    session.execute("CREATE TABLE section (course_id text, sec_id text, semester text, year text, building text, room_number text, time_slot_id text, title text, PRIMARY KEY ((course_id), sec_id, semester, year));")
    session.execute("CREATE INDEX ON section(semester);")
    
    # students_course
    session.execute("CREATE TABLE students_course (dept_name text, building text, budget text, course_id text, title text, credits text, id text, name text, tot_cred  text, PRIMARY KEY ((name), title, dept_name));")
    session.execute("CREATE INDEX ON students_course(title);")

    # students_teaches
    session.execute("CREATE TABLE students_teaches (id_instructor text, name text, salary text, id_student text, name_student text, dept_name text, tot_cred text, PRIMARY KEY ((name),  name_student, dept_name));")

#### Insert

In [4]:
def completar_tabelas_teste():
    # student e instructor
    session.execute("INSERT INTO cc6240.student  (name, dept_name, tot_cred, advisor) values('Schrefl', 'History', 4, 'Moreira');")
    session.execute("INSERT INTO cc6240.instructor (name, dept_name, salary) VALUES('McKinnon', 'Cybernetics', 94333.99);")

    

#### Delete

In [5]:
def limpar_tabela(tabela):
    if tabela == 'student':
        students = session.execute('SELECT name FROM student;')
        for s in students:
            session.execute(f"DELETE FROM {tabela} WHERE name = '{s[0]}';")

    elif tabela == 'instructor':
        instructor = session.execute('SELECT name FROM instructor;')
        for i in instructor:
            session.execute(f"DELETE FROM {tabela} WHERE name = '{i[0]}';")
    
    elif tabela == 'course_department':
        course_department = session.execute('SELECT dept_name FROM course_department;')
        for i in course_department:
            session.execute(f"DELETE FROM {tabela} WHERE dept_name = '{i[0]}';")
    
    elif tabela == 'section':
        course_id = session.execute('SELECT course_id FROM section;')
        for i in course_id:
            session.execute(f"DELETE FROM {tabela} WHERE course_id = '{i[0]}';")
            
    elif tabela == 'students_course':
        name = session.execute('SELECT name FROM students_course;')
        for i in name:
            session.execute(f"DELETE FROM {tabela} WHERE name = '{i[0]}';")

    elif tabela == 'students_teaches':
        name = session.execute('SELECT name FROM students_teaches;')
        for i in name:
            session.execute(f"DELETE FROM {tabela} WHERE name = '{i[0]}';")

def deletar_tabela(tabela):
    try:
        session.execute(f'DROP TABLE {tabela};')
    except:
        print(f'Tabela "{tabela}" não encontrada!')
        pass

#### Outras funções

In [6]:
def listar_colunas(tabela):
    instructor = session.execute(f'select * from {tabela};')
    for i in instructor:
        print(i)

def selecionar_instrutor(nomes=[]):
    prepared_statement = session.prepare("select * from instructor where name = ?;")
    for i in nomes:
        intructor = session.execute(prepared_statement, [i]).one()
        print(intructor)

def inserir_dados(name, dept_name, advisor, tot_cred):
    session.execute(f"INSERT INTO student (name, dept_name, advisor, tot_cred) VALUES ('{name}', '{dept_name}', '{advisor}', {tot_cred});")

### Conectando ao Postgres

In [7]:
url_object = URL.create(
    "postgresql+pg8000",
    username="dcxhxgql",
    password="bPlk_dl7Xc4l0WEaPCJIYR4dnO9kGZbz",
    host="motty.db.elephantsql.com",
    database="dcxhxgql",
)

engine = create_engine(url_object)
engine = create_engine(url_object)
Session = sessionmaker(bind=engine)
session_postgres = Session()
conn = engine.connect()

### Configurando as tabelas do Cassandra

In [20]:
deletar_tabela('student')
deletar_tabela('instructor')
deletar_tabela('course_department')
deletar_tabela('section')
deletar_tabela('students_course')
deletar_tabela('students_teaches')
deletar_tabela('course')

In [21]:
criar_tabelas()

#### Inserindo dados nas tabelas

In [10]:
limpar_tabela('course_department')

query = """
SELECT 
    *
FROM
    department d 
JOIN
    course c on d.dept_name = c.dept_name;
"""
cursos = conn.execute(text(query)).all()

cursos_dept_cassandra = []
		
for curso in cursos:
    query = f"INSERT INTO course_department(dept_name, building, budget, course_id, title, credits) VALUES (%s, %s, %s, %s, %s, %s);"
    session.execute(query, (curso[0], curso[1], curso[2], curso[3], curso[4], curso[6]))
    cursos_dept_cassandra.append(curso)

session_postgres.close()

In [22]:
limpar_tabela('section')

query = """
SELECT 
    s.course_id,
    s.sec_id,
    s.semester,
    s.year,
    s.building,
    s.room_number,
    s.time_slot_id,
    c.title
FROM
    section s
JOIN
    course c ON s.course_id = c.course_id;
"""

sections = conn.execute(text(query)).all()

section_cassandra = []
	
for section in sections:
    query = f"INSERT INTO section(course_id, sec_id, semester, year, building, room_number, time_slot_id, title) VALUES (%s, %s, %s, %s, %s, %s, %s, %s);"
    session.execute(query, (section[0], section[1], section[2], section[3], section[4], section[5], section[6], section[7]))
    section_cassandra.append(section)

session_postgres.close()

In [12]:
limpar_tabela('students_course')

query = """
SELECT
    d.dept_name,
    d.building,
    d.budget,
    c.course_id,
    c.title,
    c.credits,
    s.id,
    s.name,
    s.tot_cred 
FROM
    department d  
JOIN
    course c on d.dept_name = c.dept_name 
JOIN
    student s on c.dept_name = s.dept_name;
"""
stud_course = conn.execute(text(query)).all()

stud_course_cassandra = []
		
for stud in stud_course:
    query = f"INSERT INTO students_course(dept_name, building, budget, course_id, title, credits, id, name, tot_cred) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);"
    session.execute(query, (stud[0], stud[1], stud[2], stud[3], stud[4], stud[5], stud[6], stud[7], stud[8]))
    stud_course_cassandra.append(stud)

session_postgres.close()

In [13]:
limpar_tabela('students_teaches')

query = """
SELECT
    i.id AS id_instructor,
    i.name,
    i.salary,
    s.id AS id_student,
    s.name,
    s.dept_name,
    s.tot_cred
FROM
    instructor i
JOIN
    student s on i.dept_name = s.dept_name;
"""
stud_teach = conn.execute(text(query)).all()

stud_teach_cassandra = []
		
for stud in stud_teach:
    query = f"INSERT INTO students_teaches(id_instructor, name, salary, id_student, name_student, dept_name, tot_cred) VALUES (%s, %s, %s, %s, %s, %s, %s);"
    session.execute(query, (stud[0], stud[1], stud[2], stud[3], stud[4], stud[5], stud[6]))
    stud_teach_cassandra.append(stud)

session_postgres.close()

### Questões

#### Questão 1 - Listar todos os cursos oferecidos por um determinado departamento

In [14]:
def cursos_por_departamento(dept):
    cursos = session.execute(f"SELECT course_id, title, credits FROM course_department WHERE dept_name = '{dept}';")
    
    # Criar o DataFrame diretamente
    df = pd.DataFrame(cursos, columns=['course_id', 'title', 'credits'])
    
    return df

# Chamando a função e exibindo o DataFrame
df = cursos_por_departamento('Comp. Sci.')
df

,course_id,title,credits
0,CS-101,Intro. to Computer Science,4
1,CS-190,Game Design,4
2,CS-315,Robotics,3
3,CS-319,Image Processing,3
4,CS-347,Database System Concepts,3


#### Questão 2 - Recuperar todas as disciplinas de um curso específico em um determinado semestre

In [28]:
import pandas as pd

def disciplinas_por_curso_e_semestre(curso, semestre):
    # Consulta SQL para recuperar as disciplinas de um curso específico em um semestre específico
    query = f"""
    SELECT course_id, semester, title, year
    FROM section
    WHERE course_id = '{curso}' AND semester = '{semestre}';
    """
    
    # Executar a consulta SQL
    disciplinas = session.execute(query)
    
    # Criar o DataFrame diretamente
    df = pd.DataFrame(disciplinas, columns=['course_id', 'title', 'semester', 'year'])
    
    return df

# Chamando a função para recuperar as disciplinas do curso CS-101 no semestre Fall de 2017
df = disciplinas_por_curso_e_semestre('CS-101', 'Fall')
df


,course_id,title,semester,year
0,CS-101,Fall,Intro. to Computer Science,2017


In [16]:
# Recuperar todas as disciplinas de um curso específico em um determinado semestre

# section = session.execute("SELECT * FROM section;")
# for s in section:
#     print(s)

# Row(course_id='CS-347', sec_id='1', semester='Fall', year='2017', building='Taylor', room_number='3128', time_slot_id='A')
# Row(course_id='MU-199', sec_id='1', semester='Spring', year='2018', building='Packard', room_number='101', time_slot_id='D')
# Row(course_id='CS-101', sec_id='1', semester='Fall', year='2017', building='Packard', room_number='101', time_slot_id='H')
# Row(course_id='CS-101', sec_id='1', semester='Spring', year='2018', building='Packard', room_number='101', time_slot_id='F')
# Row(course_id='FIN-201', sec_id='1', semester='Spring', year='2018', building='Packard', room_number='101', time_slot_id='B')
# Row(course_id='CS-319', sec_id='1', semester='Spring', year='2018', building='Watson', room_number='100', time_slot_id='B')
# Row(course_id='CS-319', sec_id='2', semester='Spring', year='2018', building='Taylor', room_number='3128', time_slot_id='C')
# Row(course_id='PHY-101', sec_id='1', semester='Fall', year='2017', building='Watson', room_number='100', time_slot_id='A')
# Row(course_id='BIO-101', sec_id='1', semester='Summer', year='2017', building='Painter', room_number='514', time_slot_id='B')
# Row(course_id='CS-315', sec_id='1', semester='Spring', year='2018', building='Watson', room_number='120', time_slot_id='D')
# Row(course_id='HIS-351', sec_id='1', semester='Spring', year='2018', building='Painter', room_number='514', time_slot_id='C')
# Row(course_id='BIO-301', sec_id='1', semester='Summer', year='2018', building='Painter', room_number='514', time_slot_id='A')
# Row(course_id='EE-181', sec_id='1', semester='Spring', year='2017', building='Taylor', room_number='3128', time_slot_id='C')
# Row(course_id='CS-190', sec_id='1', semester='Spring', year='2017', building='Taylor', room_number='3128', time_slot_id='E')
# Row(course_id='CS-190', sec_id='2', semester='Spring', year='2017', building='Taylor', room_number='3128', time_slot_id='A')